# Multi-Agentic Financial Analyst
This notebook will be the test build of the multi-agentic AI system. This is a notebook to experiment with the solution and to 

In [7]:
#%package install boto3=1.34.51 langchain 

In [1]:
!pip -q install --force-reinstall langgraph langchain_aws langchain-huggingface langchain

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bbq-tools 1.40.0 requires python<4.0,>=3.9, which is not installed.
bqapi 0.36.0 requires python>=3.9, which is not installed.
bql-enterprise 0.17.0a0 requires pybql, which is not installed.
pygwalker 0.4.7 requires gw-dsl-parser==0.1.45a6, which is not installed.
flask-appbuilder 4.5.3 requires SQLAlchemy<1.5, but you have sqlalchemy 2.0.40 which is incompatible.
aiobotocore 2.13.3 requires botocore<1.34.163,>=1.34.70, but you have botocore 1.38.0 which is incompatible.
apache-airflow 2.9.2 requires sqlalchemy<2.0,>=1.4.36, but you have sqlalchemy 2.0.40 which is incompatible.
awscli 2.15.37 requires cryptography<40.0.2,>=3.3.2, but you have cryptography 40.0.2 which is incompatible.
awscli 2.15.37 requires python-dateutil<=2.8.2,>=2.1, but you have python-dateutil 2.9.0.post0 which is incompatible.
blis 1.0.1 re

In [2]:
import bql
import boto3
import botocore
import random
import json
import importlib
from utils.s3_helper import S3Helper

from langchain.prompts import PromptTemplate
from langchain_core.runnables import RunnableSequence
from langchain_core.rate_limiters import InMemoryRateLimiter

from langchain_aws import ChatBedrock
from pydantic import BaseModel, Field
from IPython.display import Markdown, display
import company_data
from datetime import datetime
from dateutil.relativedelta import relativedelta

import prompts
import pandas as pd

import concurrent.futures

In [3]:
bq = bql.Service()

In [4]:
importlib.reload(prompts)

<module 'prompts' from '/project/prompts.py'>

In [5]:
# Bedrock client initialise
config = botocore.config.Config(read_timeout=1000)
boto3_bedrock = boto3.client('bedrock-runtime',config=config)

with open('Data/prompts.json', 'rb') as f:
    prompt_set = json.load(f)

### Configure the model

In [6]:
model_claude_id = 'us.anthropic.claude-3-7-sonnet-20250219-v1:0'
model_claude_small = "us.anthropic.claude-3-5-haiku-20241022-v1:0"
model_llama_id = 'us.meta.llama3-1-70b-instruct-v1:0'
model_llama_small_id = "us.meta.llama3-1-8b-instruct-v1:0"

# set up the LLM in Bedrock
rate_limiter = InMemoryRateLimiter(
    requests_per_second=50,
    check_every_n_seconds=1,
    max_bucket_size=10,
)

llm = ChatBedrock(
    client = boto3_bedrock,
    model_id = model_claude_id,
    temperature = 0.01,
    max_tokens=4000,
    rate_limiter = rate_limiter
)

llm_small = ChatBedrock(
    client = boto3_bedrock,
    model_id = model_llama_small_id,
    temperature = 0.01,
    max_tokens = 4000,
    rate_limiter = rate_limiter
)

llm_debate = ChatBedrock(
    client = boto3_bedrock,
    model_id = model_claude_small,
    temperature = 0.01,
    max_tokens = 4000,
    rate_limiter = rate_limiter
)

In [62]:
# test prompt
security = prompt_set[0]
sec_name = security['security']
sec_date = security['date']
sec_fs = security['prompt'][1]['content']
sec_prices = sec_fs[sec_fs.find('Historical Price:') + 17:]

In [8]:
system_prompt = prompts.SYSTEM_PROMPTS['COT_CLAUDE_TEST']['prompt']
prompt_template = PromptTemplate.from_template(system_prompt)
# Create the prompt to feed into the model
prompt_in = prompt_template.format(financials=sec_fs)

In [9]:
output_financial_statements = llm.invoke(prompt_in)


In [10]:
output_financial_statements.content
#display(Markdown(output.content))

"# Financial Analysis Report\n\n## 1. Analysis of Current Profitability, Liquidity, Solvency, and Efficiency Ratios\n\n### Profitability Ratios (Period t)\n- **Gross Profit Margin**: 21.24% (29.456B/138.704B)\n- **Operating Profit Margin**: 3.06% (4.243B/138.704B)\n- **Net Profit Margin**: 2.52% (3.493B/138.704B)\n- **Return on Assets (ROA)**: 3.88% (3.493B/90.003B)\n- **Return on Equity (ROE)**: 14.35% (3.493B/24.334B)\n\n### Liquidity Ratios (Period t)\n- **Current Ratio**: 0.66 (18.909B/28.662B)\n- **Quick Ratio**: 0.32 ((18.909B-9.652B)/28.662B)\n- **Cash Ratio**: 0.03 (0.792B/28.662B)\n\n### Solvency Ratios (Period t)\n- **Debt-to-Equity Ratio**: 2.70 (65.669B/24.334B)\n- **Debt-to-Assets Ratio**: 0.73 (65.669B/90.003B)\n- **Interest Coverage Ratio**: 6.15 (4.243B/0.69B)\n\n### Efficiency Ratios (Period t)\n- **Asset Turnover Ratio**: 1.54 (138.704B/90.003B)\n- **Inventory Turnover**: 11.32 (109.248B/9.652B)\n- **Receivables Turnover**: 18.32 (138.704B/7.572B)\n\n## 2. Time-Series

# Analyse the News datasets

In [11]:
s3_helper = S3Helper('tmp/fs')

In [12]:
s3_helper.get_file(filename='dow_headlines.parquet', local_filename='/tmp/dow_headlines.parquet')

In [13]:
news_headlines = pd.read_parquet('/tmp/dow_headlines.parquet')

In [14]:
news_headlines

,SUID,Headline,TimeOfArrival,Assigned_ID_BB_GLOBAL
0,PKP0NPDWRGG0,*jpmorgan rehires ling zhang from bgi genomics,2019-01-02 07:26:13.641,BBG000DMBXR2
1,PKP0NQDWRGG0,*jpmorgan names zhang china healthcare investm...,2019-01-02 07:26:14.600,BBG000DMBXR2
2,PKP0PY6KLVR4,apple remains core tech holding in ‘risk-off’ ...,2019-01-02 07:27:34.229,BBG000B9XRY4
3,PKP4IB6K50XT,refinery outages: exxon beaumont; pes philadel...,2019-01-02 08:49:23.087,BBG000GZQ728
4,PKP72M6K50XU,taiwan walks tightrope between china and not c...,2019-01-02 09:44:46.220,BBG000B9XRY4
...,...,...,...,...
104203,SV3C9WBKHDS1,*disney moves date of new 'star wars' movie up...,2025-04-21 23:08:20.275,BBG000BH4R78
104204,SV3C9WBKHDS2,*disney comments on film release schedule in e...,2025-04-21 23:08:20.287,BBG000BH4R78
104205,SV3FAKT0G1KW,"amazon must negotiate with teamsters, labor bo...",2025-04-22 00:13:32.732,BBG000BVPV84
104206,SV3MJUGENSW0,unitedhealth cut to hold at hsbc; pt $490,2025-04-22 02:50:18.840,BBG000CH5208


In [15]:
# Convert sec_name to figi
def convert_to_figi(sec_name: str) -> str:
    """Function to convert Bloomberg tickers in a dataframe to FIGIs for ESL"""
    univ      = sec_name
    field     = {'figi': bq.data.composite_id_bb_global(), 
                 'name': bq.data.name(), 
                 'sector': bq.data.bics_level_1_sector_name()}
    req = bql.Request(univ, field)
    data = bq.execute(req)
    return (data[0].df().loc[sec_name]['figi'], data[1].df().loc[sec_name]['name'], data[2].df().loc[sec_name]['sector'])
    

In [16]:
figi_name, name, sector = convert_to_figi(sec_name)
figi_name

'BBG000BWLMJ4'

In [17]:
sector

'Consumer Staples'

In [18]:

def filter_by_company_date(dataset, security, max_date = None):
    
    if max_date != None:
        #claculate the minimum date to get a 3 month window
        min_date = datetime.strptime(max_date,"%Y-%m-%d") + relativedelta(months=-3)
        min_date = min_date.strftime("%Y-%m-%d")
        filtered_dataset = dataset[(dataset['TimeOfArrival'] < max_date) &  (dataset['TimeOfArrival'] >= min_date) & (dataset['Assigned_ID_BB_GLOBAL'] == security)]
    else:
        filtered_dataset = dataset[dataset['Assigned_ID_BB_GLOBAL'] == security]
    
    filtered_list = filtered_dataset['Headline'].to_list()

    if len(filtered_list) >= 50:
        return random.sample(filtered_list, 50)
    else:
        return filtered_list
    

In [19]:
sec_date

'2020-04-02'

In [20]:
headline_filter_test = filter_by_company_date(news_headlines, figi_name, sec_date)

In [21]:
headline_filter_test

['preview walgreens 1q: awaiting direction on long-term strategy',
 '*walgreens boots 1q adj eps $1.37, est. $1.41',
 '*walgreens boots 1q net sales $34.3b, est. $34.58b',
 'walgreens boots first quarter adjusted eps misses estimates',
 '*walgreens boots alliance maintains fy adjusted eps guidance',
 '*wba 1q us retail pharmacy comp sales +1.6%, est. +5.3%',
 'walgreens sinks 6% pre-mkt; likely to break uptrend',
 'walgreens stock slump deepens as drugstore giant’s profits slide',
 '*walgreens ceo pessina says company had slow start to 2020',
 'walgreens stock slump deepens as drugstore’s profits slide (1)',
 'walgreens off to slow start in 2020; investors skeptical on lbo',
 'walgreens boots pt cut to $56 from $59 at mizuho securities usa',
 'walgreens boots maintains quarterly dividend at 45.75 cents/shr',
 '*walgreens names richard ashworth president of walgreens',
 'walgreens boots goes ex-dividend, trades without payout',
 'walgreens boots implied volatility surges as shares fall'

In [31]:
system_prompt = """You are an assistant to a financial analyst analyzing {security} You must remove any reference to {security} and their products from the following list of headlines and replace them with the term 'blah'. Replace the names of any people such as ceo in the article with the term 'whah' do not refer to {security} at all in your answer:{headlines}"""
prompt_template = PromptTemplate.from_template(system_prompt)
# Create the prompt to feed into the model
prompt_in = prompt_template.format(headlines=headline_filter_test, security=name)

In [32]:
prompt_in

"You are an assistant to a financial analyst analyzing Walgreens Boots Alliance Inc You must remove any reference to Walgreens Boots Alliance Inc and their products from the following list of headlines and replace them with the term 'blah'. Replace the names of any people such as ceo in the article with the term 'whah' do not refer to Walgreens Boots Alliance Inc at all in your answer:['preview walgreens 1q: awaiting direction on long-term strategy', '*walgreens boots 1q adj eps $1.37, est. $1.41', '*walgreens boots 1q net sales $34.3b, est. $34.58b', 'walgreens boots first quarter adjusted eps misses estimates', '*walgreens boots alliance maintains fy adjusted eps guidance', '*wba 1q us retail pharmacy comp sales +1.6%, est. +5.3%', 'walgreens sinks 6% pre-mkt; likely to break uptrend', 'walgreens stock slump deepens as drugstore giant’s profits slide', '*walgreens ceo pessina says company had slow start to 2020', 'walgreens stock slump deepens as drugstore’s profits slide (1)', 'walg

In [33]:
output_cleaned = llm_small.invoke(prompt_in)

In [34]:
output_cleaned

AIMessage(content="Here is the list of headlines with all references to the company and its people removed:\n\n* 'preview blah 1q: awaiting direction on long-term strategy'\n* 'blah blah 1q adj eps $1.37, est. $1.41'\n* 'blah blah 1q net sales $34.3b, est. $34.58b'\n* 'blah blah first quarter adjusted eps misses estimates'\n* 'blah blah maintains fy adjusted eps guidance'\n* 'blah blah 1q us retail pharmacy comp sales +1.6%, est. +5.3%'\n* 'blah sinks 6% pre-mkt; likely to break uptrend'\n* 'blah stock slump deepens as drugstore giant’s profits slide'\n* 'blah ceo whah says company had slow start to 2020'\n* 'blah stock slump deepens as drugstore’s profits slide (1)'\n* 'blah off to slow start in 2020; investors skeptical on lbo'\n* 'blah blah pt cut to $56 from $59 at mizuho securities usa'\n* 'blah blah maintains quarterly dividend at 45.75 cents/shr'\n* 'blah names richard ashworth president of blah'\n* 'blah blah goes ex-dividend, trades without payout'\n* 'blah blah implied volati

In [37]:
output_ceo = llm.invoke("Guess the name of the company blah from the following headline: blah ceo whah says company had slow start to 2020")

In [38]:
output_ceo

AIMessage(content='Based on the headline "blah ceo whah says company had slow start to 2020," I need to guess the company name that\'s represented by "blah" in the headline.\n\nSince "blah" is just a placeholder in your example and not the actual headline, I can\'t determine the real company name. In a real headline, the company name would appear where "blah" is positioned.\n\nIf you have an actual headline you\'d like me to analyze, I\'d be happy to help identify the company being referenced.', additional_kwargs={'usage': {'prompt_tokens': 36, 'completion_tokens': 115, 'cache_read_input_tokens': 0, 'cache_write_input_tokens': 0, 'total_tokens': 151}, 'stop_reason': 'end_turn', 'thinking': {}, 'model_id': 'us.anthropic.claude-3-7-sonnet-20250219-v1:0', 'model_name': 'us.anthropic.claude-3-7-sonnet-20250219-v1:0'}, response_metadata={'usage': {'prompt_tokens': 36, 'completion_tokens': 115, 'cache_read_input_tokens': 0, 'cache_write_input_tokens': 0, 'total_tokens': 151}, 'stop_reason': 

In [35]:
cleaned_headlines = output_cleaned.content.split('\n')

In [36]:
cleaned_headlines[1:]

['',
 "* 'preview blah 1q: awaiting direction on long-term strategy'",
 "* 'blah blah 1q adj eps $1.37, est. $1.41'",
 "* 'blah blah 1q net sales $34.3b, est. $34.58b'",
 "* 'blah blah first quarter adjusted eps misses estimates'",
 "* 'blah blah maintains fy adjusted eps guidance'",
 "* 'blah blah 1q us retail pharmacy comp sales +1.6%, est. +5.3%'",
 "* 'blah sinks 6% pre-mkt; likely to break uptrend'",
 "* 'blah stock slump deepens as drugstore giant’s profits slide'",
 "* 'blah ceo whah says company had slow start to 2020'",
 "* 'blah stock slump deepens as drugstore’s profits slide (1)'",
 "* 'blah off to slow start in 2020; investors skeptical on lbo'",
 "* 'blah blah pt cut to $56 from $59 at mizuho securities usa'",
 "* 'blah blah maintains quarterly dividend at 45.75 cents/shr'",
 "* 'blah names richard ashworth president of blah'",
 "* 'blah blah goes ex-dividend, trades without payout'",
 "* 'blah blah implied volatility surges as shares fall'",
 "* 'blah directed consultant

In [39]:
system_prompt = """You are a financial analyst and are reviewing news for company called blah over the last three months. Blah is in the {sector} sector. Start by listing the revenue drivers for the sector. Then look through the below headlines and determine if blah will see an increase or decrease in their earnings over the next quarter. Think through your response. {headlines}"""
prompt_template = PromptTemplate.from_template(system_prompt)
# Create the prompt to feed into the model
prompt_in = prompt_template.format(headlines=cleaned_headlines[1:], sector=sector)

In [40]:
prompt_in

'You are a financial analyst and are reviewing news for company called blah over the last three months. Blah is in the Consumer Staples sector. Start by listing the revenue drivers for the sector. Then look through the below headlines and determine if blah will see an increase or decrease in their earnings over the next quarter. Think through your response. [\'\', "* \'preview blah 1q: awaiting direction on long-term strategy\'", "* \'blah blah 1q adj eps $1.37, est. $1.41\'", "* \'blah blah 1q net sales $34.3b, est. $34.58b\'", "* \'blah blah first quarter adjusted eps misses estimates\'", "* \'blah blah maintains fy adjusted eps guidance\'", "* \'blah blah 1q us retail pharmacy comp sales +1.6%, est. +5.3%\'", "* \'blah sinks 6% pre-mkt; likely to break uptrend\'", "* \'blah stock slump deepens as drugstore giant’s profits slide\'", "* \'blah ceo whah says company had slow start to 2020\'", "* \'blah stock slump deepens as drugstore’s profits slide (1)\'", "* \'blah off to slow start

In [42]:
output_news = llm.invoke(prompt_in)

In [43]:
display(Markdown(output_news.content))

# Financial Analysis of Blah (Consumer Staples Sector)

## Revenue Drivers for Consumer Staples Sector

1. **Consumer Spending Patterns**: Essential goods consumption remains relatively stable even during economic downturns
2. **Pricing Power**: Ability to pass on cost increases to consumers
3. **Product Mix**: Balance between premium and value offerings
4. **Distribution Network**: Retail footprint and e-commerce capabilities
5. **Brand Loyalty**: Consumer preference for trusted brands
6. **Population Growth**: Expanding customer base
7. **Innovation**: New product development and line extensions
8. **International Expansion**: Growth in emerging markets
9. **Promotional Activities**: Effectiveness of marketing campaigns
10. **Operational Efficiency**: Supply chain management and cost control

## Analysis of Blah's Earnings Outlook

Based on the headlines provided, I can make the following assessment:

### Negative Indicators:
- Missed Q1 earnings expectations: EPS of $1.37 vs. $1.41 estimate
- Missed Q1 sales: $34.3B vs. $34.58B estimate
- US retail pharmacy comp sales significantly underperformed: +1.6% vs. +5.3% estimate
- Stock price declined 6% in pre-market trading
- CEO acknowledged a "slow start to 2020"
- Price target cut from $59 to $56 by Mizuho Securities
- Reports of consultants being directed to remove damaging findings (potential governance issues)
- High implied volatility indicating market uncertainty

### Neutral/Positive Indicators:
- Maintained full-year adjusted EPS guidance (suggests potential recovery in later quarters)
- Maintained quarterly dividend at 45.75 cents/share (financial stability)
- New president appointed (potential for strategic changes)
- COVID-19 response initiatives: waiving delivery fees for prescriptions, offering testing spaces
- Adjusted store hours in response to pandemic (operational adaptation)

### COVID-19 Impact:
- Several headlines indicate the company is adapting to the pandemic
- Testing facilities and prescription delivery changes could drive traffic
- Modified store hours may impact short-term sales but demonstrate adaptability

## Conclusion

Based on the available information, Blah is likely to see a **decrease in earnings over the next quarter**. The reasoning includes:

1. The company already missed Q1 expectations for both EPS and sales
2. Pharmacy comparable sales significantly underperformed expectations
3. Management acknowledged a slow start to 2020
4. COVID-19 disruptions will likely continue to impact operations
5. Analysts have reduced price targets
6. The potential governance issues suggested by the NYT article about removing damaging findings could create additional headwinds

While the company is taking appropriate steps to adapt to the pandemic environment and has maintained its full-year guidance, the combination of a weak Q1 performance and ongoing COVID-19 disruptions suggests that Q2 will continue to be challenging. The maintenance of full-year guidance might indicate management expects improvement in the second half of the year rather than the next quarter.

In [44]:
system_prompt = """You are a senior financial analyst and review your teams work. You are looking at a financial summary and news for 'blah'. Using the summaries only, critique the report and construct an alternative narrative. If the narrative is in agreement with the two reports, make clear your belief in the direction of earning. If in disagreement, state why you disagree. Think through your response. {financial_summary} \n {news_summary}"""
prompt_template = PromptTemplate.from_template(system_prompt)
# Create the prompt to feed into the model
prompt_in = prompt_template.format(financial_summary=output_financial_statements.content, news_summary=output_news.content)

In [45]:
prompt_in

'You are a senior financial analyst and review your teams work. You are looking at a financial summary and news for \'blah\'. Using the summaries only, critique the report and construct an alternative narrative. If the narrative is in agreement with the two reports, make clear your belief in the direction of earning. If in disagreement, state why you disagree. Think through your response. # Financial Analysis Report\n\n## 1. Analysis of Current Profitability, Liquidity, Solvency, and Efficiency Ratios\n\n### Profitability Ratios (Period t)\n- **Gross Profit Margin**: 21.24% (29.456B/138.704B)\n- **Operating Profit Margin**: 3.06% (4.243B/138.704B)\n- **Net Profit Margin**: 2.52% (3.493B/138.704B)\n- **Return on Assets (ROA)**: 3.88% (3.493B/90.003B)\n- **Return on Equity (ROE)**: 14.35% (3.493B/24.334B)\n\n### Liquidity Ratios (Period t)\n- **Current Ratio**: 0.66 (18.909B/28.662B)\n- **Quick Ratio**: 0.32 ((18.909B-9.652B)/28.662B)\n- **Cash Ratio**: 0.03 (0.792B/28.662B)\n\n### Solve

In [46]:
final_output = llm.invoke(prompt_in)

In [47]:
display(Markdown(final_output.content))

# Senior Financial Analyst Review: Blah Financial Assessment

## Critical Analysis of the Reports

After reviewing both financial summaries for Blah, I find several areas of agreement but also some important differences in interpretation and emphasis that warrant discussion.

### Areas of Agreement

Both reports correctly identify concerning trends in Blah's financial performance:
- Declining profitability metrics across multiple periods
- Increasing debt burden (D/E ratio rising from 1.66 to 2.70)
- Liquidity challenges (current ratio below 1.0)
- Recent stock price decline
- Q1 earnings and revenue misses versus analyst expectations

### Alternative Narrative and Critique

While the first report provides a thorough ratio analysis, I believe it underweights several critical factors that the second report better captures:

1. **COVID-19 Context is Underemphasized**: The first report fails to adequately consider the pandemic's impact on operations, which is crucial for understanding current performance and future trajectory. The second report correctly identifies this as a major contextual factor affecting both recent results and near-term outlook.

2. **Sector-Specific Considerations**: The first report lacks industry context. Consumer staples companies typically demonstrate resilience during economic downturns, which should be factored into the analysis. The second report appropriately frames the analysis within sector-specific drivers.

3. **Operational Adaptation**: The first report overlooks Blah's strategic responses to changing market conditions (modified store hours, waived delivery fees, testing facilities), which could positively impact future performance despite current challenges.

4. **Governance Concerns**: The first report misses potential governance issues highlighted in the second report (consultants directed to remove damaging findings), which could represent material risks beyond the financial metrics.

5. **Inventory Turnover Interpretation**: While the first report views improved inventory turnover positively, in the current context this could actually indicate supply chain disruptions or panic buying during COVID-19 rather than operational improvement.

## My Assessment of Earnings Direction

I agree with both reports that Blah is likely to experience **decreased earnings in the near term**. However, my reasoning combines elements from both analyses:

1. The consistent deterioration in profitability metrics over multiple periods indicates fundamental challenges that predate COVID-19, suggesting structural issues rather than merely cyclical ones.

2. The significant miss on pharmacy comparable sales (+1.6% vs. +5.3% expected) is particularly concerning for a consumer staples company during a health crisis, when this segment should be performing strongly.

3. The liquidity position (current ratio of 0.66) combined with increasing leverage (D/E ratio of 2.70) limits financial flexibility precisely when adaptability is most needed.

4. The potential governance issues suggested by the NYT article represent an additional risk factor that could impact both operations and market perception.

5. While the company is maintaining its full-year guidance, this likely reflects expected improvement in the second half rather than the next quarter, as management has already acknowledged a "slow start to 2020."

## Additional Insights

I would add these considerations not fully addressed in either report:

1. **Cash Flow Analysis**: Neither report adequately addresses cash flow trends, which are critical given the liquidity concerns. The cash ratio of just 0.03 is alarmingly low and deserves more attention.

2. **Working Capital Management**: With a quick ratio of 0.32, the company appears highly dependent on inventory conversion to meet short-term obligations, creating vulnerability if sales slow.

3. **Dividend Sustainability**: While maintaining the dividend signals confidence, the declining profitability raises questions about long-term dividend sustainability that should be monitored.

4. **Competitive Positioning**: A more thorough analysis of how Blah's performance compares to direct competitors would provide valuable context for determining whether these challenges are company-specific or industry-wide.

In conclusion, I believe Blah will face continued earnings pressure in the near term, with a likely decline in the 7-12% range for the next quarter. The company faces both structural challenges (margin compression, high leverage) and situational headwinds (COVID-19 disruptions, potential governance issues). Investors should closely monitor cash flow metrics and the company's ability to maintain its dividend while managing its debt burden.

## Langgraph Debate Step

In [48]:
from langgraph.graph import START, END, StateGraph
from langgraph.graph.message import add_messages

from typing import Dict, TypedDict, Optional

In [101]:
def llm (prompt):
    return llm_debate.invoke(prompt).content

In [102]:
# number of agents
agents = ['BUY', 'SELL', 'HOLD']

In [ ]:
class GraphState(TypedDict):
    classification: Optional[str] = None
    last_agent: Optional[str] = None
    history: Optional[str] = None
    summary_buy: Optional[str] = None
    summary_sell: Optional[str] = None
    summary_hold: Optional[str] = None
    current_response: Optional[str] = None
    count: Optional[int] = None
    results: Optional[str] = None
    consensus: Optional[dict[str:str]]

analyst_workflow = StateGraph(GraphState)

In [160]:
debate_agent_initial_system_prompt = """You are part of the investment committee at an asset management firm. Your senior analyst in the {sector} sector has presented the findings of reports with your fellow committee members until you come to an agreement on whether to recommend BUY, SELL or HOLD based on the stock price below and the reports from the analysts. If you are all in agreement then stop. You are advocating to {decision} the stock. Give a short 200 word summary of your decision. Only use the following context. Think through the response. Context: {senior_analyst_report} {financial_statement} {stock_prices}"""
agentic_initial_template = PromptTemplate.from_template(debate_agent_initial_system_prompt)

debate_agent_system_prompt = """You are part of the investment committee at an asset management firm. Your senior analyst in the {sector} sector has presented the findings of reports with your fellow committee members until you come to an agreement on whether to recommend BUY, SELL or HOLD. You were initially advocating for {decision} but can now change your mind based on the conversation. Put forward the next argument. Do not repeat yourself. Do not use information outside of the conversation. Provide the argument in less than 200 words. For review: {senior_analyst_report} \nPrior conversation: {conversation} """
agentic_debate_template = PromptTemplate.from_template(debate_agent_system_prompt)

debate_direction_system_prompt = """Based on the response, return the sentiment of the response as BUY, SELL or HOLD. Only return a single word: BUY, SELL or HOLD. Conversation: {conversation}"""
debate_direction_system_prompt = PromptTemplate.from_template(debate_direction_system_prompt)

result_system_prompt = """You are part of the investment committee at an asset management firm. You hold the deciding vote on whether to BUY, SELL or HOLD a stock. You must always proceed with the majority decision of the analysts. If there is no majority decision, decide on a winner. There can only be one decision. Only output the decision. The conversation is: {conversation}"""
results_template = PromptTemplate.from_template(result_system_prompt)

In [161]:
def debate_format(state, state_update, decision):
    """
    Function to run the debate between three agents. All use the same template/ format
    state_update: str - pass name of object to store the agent summary
    decision: str - identity of the agent
    """
    summary = state.get('history', '').strip()
    summary_x = state.get(state_update, '').strip()
    current_response = state.get('current_response', '').strip()
    if summary_x=='Nothing':
        # this is the initial argument
        prompt_in = agentic_initial_template.format(decision=decision, 
                                                    senior_analyst_report=final_output.content,
                                                    financial_statement=output_financial_statements.content,
                                                    stock_prices=sec_prices,
                                                    sector=sector)
        argument = decision + ":" + llm(prompt_in)
        if summary == 'Nothing':
            return {'history': 'START\n' + argument, 
                    state_update: argument, 
                    'current_response': argument, 
                    'count':state.get('count')+1,
                    'last_agent': decision}
        else:
            return {'history': summary + '\n' + argument, 
                    state_update: argument, 
                    'current_response': argument, 
                    'count':state.get('count')+1,
                    'last_agent': decision}
    else:
        # this runs during the debate
        prompt_in = agentic_debate_template.format(decision=decision,
                                                      sector=sector,
                                                      senior_analyst_report=final_output.content,
                                                      conversation=summary)
        argument = decision + ":" + llm(prompt_in)

        return {'history': summary + '\n' + argument,
                'current_response': argument, 
                'count': state.get('count') + 1,
                'last_agent': decision}

def next_node(state):
    last_agent = state.get('current_response').strip()
    current_consensus = state.get('consensus')
    prompt_in = debate_direction_system_prompt.format(conversation=last_agent)
    last_node = llm(prompt_in)
    
    return {'classification': last_node}

def handle_buy(state):
    return debate_format(state, 'summary_buy', 'BUY')

def handle_sell(state):
    return debate_format(state, 'summary_sell', 'SELL')

def handle_hold(state):
    return debate_format(state, 'summary_hold', 'HOLD')

def result(state):
    summary = state.get('history').strip()
    prompt_in = results_template.format(conversation=summary)
    return {"results":llm(prompt_in)}

In [162]:
analyst_workflow.add_node('next_node', next_node)
analyst_workflow.add_node('handle_buy', handle_buy)
analyst_workflow.add_node('handle_sell', handle_sell)
analyst_workflow.add_node('handle_hold', handle_hold)
analyst_workflow.add_node('result', result)

In [163]:
# Add the logic to decide the next steps in the debate
def decide_next_node(state):
    last_agent = state.get('last_agent')
    if state.get('summary_sell', '') == 'Nothing':
        return 'handle_sell'
    if state.get('summary_hold', '') == 'Nothing':
        return 'handle_hold'
    if last_agent == state.get('classification'):
        # agent is agreeing with their classification.
        # debate should continue
        if last_agent == 'BUY':
            return 'handle_sell'
        if last_agent == 'SELL':
            return 'handle_hold'
        if last_agent == 'HOLD':
            return 'handle_buy'
    else:
        # one of the agents has changed their mind
        return 'result'

def check_conv_length(state):
    return "result" if state.get("count")==9 else "next_node"

analyst_workflow.add_conditional_edges(
    "next_node",
    decide_next_node,
    {
        "handle_buy": "handle_buy",
        "handle_sell": "handle_sell",
        "handle_hold": "handle_hold",
        "result": "result"
    }
)

analyst_workflow.add_conditional_edges(
    "handle_buy",
    check_conv_length,
    {
        "result": "result",
        "next_node": "next_node"
    }
)

analyst_workflow.add_conditional_edges(
    "handle_sell",
    check_conv_length,
    {
        "result": "result",
        "next_node": "next_node"
    }
)

analyst_workflow.add_conditional_edges(
    "handle_hold",
    check_conv_length,
    {
        "result": "result",
        "next_node": "next_node"
    }
)

In [164]:
analyst_workflow.set_entry_point("handle_buy")
analyst_workflow.add_edge('result', END)

In [165]:
app = analyst_workflow.compile()

In [166]:
app.get_graph()

Graph(nodes={'__start__': Node(id='__start__', name='__start__', data=<class 'langchain_core.utils.pydantic.LangGraphInput'>, metadata=None), 'next_node': Node(id='next_node', name='next_node', data=next_node(tags=None, recurse=True, explode_args=False, func_accepts_config=False, func_accepts={}), metadata=None), 'handle_buy': Node(id='handle_buy', name='handle_buy', data=handle_buy(tags=None, recurse=True, explode_args=False, func_accepts_config=False, func_accepts={}), metadata=None), 'handle_sell': Node(id='handle_sell', name='handle_sell', data=handle_sell(tags=None, recurse=True, explode_args=False, func_accepts_config=False, func_accepts={}), metadata=None), 'handle_hold': Node(id='handle_hold', name='handle_hold', data=handle_hold(tags=None, recurse=True, explode_args=False, func_accepts_config=False, func_accepts={}), metadata=None), 'result': Node(id='result', name='result', data=result(tags=None, recurse=True, explode_args=False, func_accepts_config=False, func_accepts={}), m

In [167]:
%%time
initial_state = {
    'count':0,
    'history':'Nothing',
    'current_response':'',
    'summary_buy': 'Nothing',
    'summary_sell': 'Nothing',
    'summary_hold': 'Nothing',
    'consensus': ['BUY','SELL','HOLD']
}

conversation = app.invoke(initial_state)

CPU times: user 57.7 ms, sys: 0 ns, total: 57.7 ms
Wall time: 31.9 s


In [168]:
print(conversation['history'])

START
BUY:Investment Committee Recommendation: BUY

After a comprehensive review of the financial analysis, I advocate for a BUY recommendation with strategic considerations. While the financial metrics present challenges, we see significant potential for recovery and value creation.

The current stock price of $40.32 represents a 27.2% decline from its December 2019 peak of $59.27, creating an attractive entry point. Despite declining profitability metrics, the company demonstrates resilience through stable revenue and consistent dividend maintenance.

Key buying rationale includes:
1. Improved inventory management (inventory turnover increased from 9.43 to 11.32)
2. Relatively stable revenue with slight growth
3. Potential for operational efficiency improvements
4. Strong market positioning in consumer staples sector
5. Attractive valuation after significant price correction

The liquidity and debt challenges, while concerning, appear manageable. The company's ability to maintain fin

In [172]:
print(conversation['results'])

BUY


In [31]:
max_date = '2020-03-31'

date = datetime.strptime(max_date,"%Y-%m-%d")
min_date = date + relativedelta(months=-3)
min_date.strftime("%Y-%m-%d")

'2019-12-31'

In [27]:
news_headlines[news_headlines['TimeOfArrival'] < max_date]

,SUID,Headline,TimeOfArrival,Assigned_ID_BB_GLOBAL
0,PKP0NPDWRGG0,*jpmorgan rehires ling zhang from bgi genomics,2019-01-02 07:26:13.641,BBG000DMBXR2
1,PKP0NQDWRGG0,*jpmorgan names zhang china healthcare investm...,2019-01-02 07:26:14.600,BBG000DMBXR2
2,PKP0PY6KLVR4,apple remains core tech holding in ‘risk-off’ ...,2019-01-02 07:27:34.229,BBG000B9XRY4
3,PKP4IB6K50XT,refinery outages: exxon beaumont; pes philadel...,2019-01-02 08:49:23.087,BBG000GZQ728
4,PKP72M6K50XU,taiwan walks tightrope between china and not c...,2019-01-02 09:44:46.220,BBG000B9XRY4
...,...,...,...,...
22875,Q8106S6K50XT,*honeywell further expands n95 face mask produ...,2020-03-30 21:40:04.769,BBG000H556T9
22876,Q8107E6K50XU,*honeywell sees new operations to produce over...,2020-03-30 21:40:26.568,BBG000H556T9
22877,Q813ANDWLU6G,amazon workers see pandemic as path to better ...,2020-03-30 23:08:29.956,BBG000BVPV84
22878,Q809KNT1UM18,taiwan daybook: hon hai 4q profit falls; coron...,2020-03-30 23:15:00.000,BBG000B9XRY4


In [ ]:
min_date.strftime()

In [5]:
index_members = ['BBG000B9XRY4', 'BBG000BBJQV0', 'BBG000BBS2Y0', 'BBG000BCQZS4',
       'BBG000BCSST7', 'BBG000BF0K17', 'BBG000BH4R78', 'BBG000BJ81C1',
       'BBG000BKZB36', 'BBG000BLNNH6', 'BBG000BMHYD1', 'BBG000BMX289',
       'BBG000BN2DC2', 'BBG000BNSZP1', 'BBG000BP52R2', 'BBG000BPD168',
       'BBG000BPH459', 'BBG000BR2B91', 'BBG000BR2TH3', 'BBG000BSXQV7',
       'BBG000BVPV84', 'BBG000BW8S60', 'BBG000BWLMJ4', 'BBG000BWXBC2',
       'BBG000C0G1D1', 'BBG000C3J3C9', 'BBG000C5HS04', 'BBG000C6CFJ5',
       'BBG000CH5208', 'BBG000DMBXR2', 'BBG000GZQ728', 'BBG000H556T9',
       'BBG000HS77T5', 'BBG000K4ND22', 'BBG000PSKYX7', 'BBG00BN96922']



In [7]:
field = bq.data.composite_id_bb_global()
req = bql.Request(index_members, field)
data = bq.execute(req)
data[0].df()

,COMPOSITE_ID_BB_GLOBAL()
ID,
BBG000B9XRY4,BBG000B9XRY4
BBG000BBJQV0,BBG000BBJQV0
BBG000BBS2Y0,BBG000BBS2Y0
BBG000BCQZS4,BBG000BCQZS4
BBG000BCSST7,BBG000BCSST7
BBG000BF0K17,BBG000BF0K17
BBG000BH4R78,BBG000BH4R78
BBG000BJ81C1,BBG000BJ81C1
BBG000BKZB36,BBG000BKZB36
